In [2]:
teams_info = ("https://www.iplt20.com/teams")

In [9]:
from bs4 import BeautifulSoup
import requests

import pandas as pd


In [4]:
# Helper Functions

def check_response(url):
    return requests.get(url).status_code



In [5]:
resp = requests.get(teams_info)
soup = BeautifulSoup(resp.text, "html.parser")

In [6]:
def get_teams_information(url):
    try:
        resp = requests.get(url)
        # resp.raise_for_status()
        print("GOT GREEN LIGHT FROM URL!!!")
    except requests.exceptions.RequestException as e:
        print("Request Faild : {e}")

    soup = BeautifulSoup(resp.text, "html.parser")
    
    l = []  # Will hold the infomation that is scrapped

    for i in soup.find_all("a", {"data-team_name":True}):
        team_name = i['data-team_name']  
        
        url = i['href']

        abbrevation = i.find('span').text

        trophy_div = i.find('div', class_='trophy-text-align')
        trophy_year =  trophy_div.text if trophy_div else None

        logo = i.find("img")['src']

        l.append([team_name, abbrevation, trophy_year, logo, url])

    df = pd.DataFrame(l, columns = ["team_name","abbrevation","trophy_year","logo","team_url"])

    return df

In [ ]:
df = get_teams_information(teams_info)

GOT GREEN LIGHT FROM URL!!!


In [10]:
df

,team_name,abbrevation,trophy_year,logo,team_url
0,Chennai Super Kings,CSK,2010 | 2011 | 2018 | 2021 | 2023,https://documents.iplt20.com/ipl/CSK/logos/Log...,https://www.iplt20.com/teams/chennai-super-kings
1,Delhi Capitals,DC,None,https://documents.iplt20.com/ipl/DC/Logos/Logo...,https://www.iplt20.com/teams/delhi-capitals
2,Gujarat Titans,GT,2022,https://documents.iplt20.com/ipl/GT/Logos/Logo...,https://www.iplt20.com/teams/gujarat-titans
3,Kolkata Knight Riders,KKR,2012 | 2014 | 2024,https://documents.iplt20.com/ipl/KKR/Logos/Log...,https://www.iplt20.com/teams/kolkata-knight-ri...
4,Lucknow Super Giants,LSG,None,https://documents.iplt20.com/ipl/LSG/Logos/Log...,https://www.iplt20.com/teams/lucknow-super-giants
5,Mumbai Indians,MI,2013 | 2015 | 2017 | 2019 | 2020,https://documents.iplt20.com/ipl/MI/Logos/Logo...,https://www.iplt20.com/teams/mumbai-indians
6,Punjab Kings,PBKS,None,https://documents.iplt20.com/ipl/PBKS/Logos/Lo...,https://www.iplt20.com/teams/punjab-kings
7,Rajasthan Royals,RR,2008,https://documents.iplt20.com/ipl/RR/Logos/Logo...,https://www.iplt20.com/teams/rajasthan-royals
8,Royal Challengers Bengaluru,RCB,None,https://documents.iplt20.com/ipl/RCB/Logos/Log...,https://www.iplt20.com/teams/royal-challengers...
9,Sunrisers Hyderabad,SRH,2016,https://documents.iplt20.com/ipl/SRH/Logos/Log...,https://www.iplt20.com/teams/sunrisers-hyderabad


In [10]:
import pandas as pd
pd.set_option("display.max_columns", None)
df = pd.read_csv("scrapped_teams_info/ipl_teams_data.csv")

mi_url = "https://www.iplt20.com/teams/mumbai-indians"
srh_url = "https://www.iplt20.com/teams/sunrisers-hyderabad"

In [3]:
list(df['team_url'])

['https://www.iplt20.com/teams/chennai-super-kings',
 'https://www.iplt20.com/teams/delhi-capitals',
 'https://www.iplt20.com/teams/gujarat-titans',
 'https://www.iplt20.com/teams/kolkata-knight-riders',
 'https://www.iplt20.com/teams/lucknow-super-giants',
 'https://www.iplt20.com/teams/mumbai-indians',
 'https://www.iplt20.com/teams/punjab-kings',
 'https://www.iplt20.com/teams/rajasthan-royals',
 'https://www.iplt20.com/teams/royal-challengers-bengaluru',
 'https://www.iplt20.com/teams/sunrisers-hyderabad']

In [16]:

resp = requests.get(mi_url)
soup = BeautifulSoup(resp.text, "html.parser")

In [17]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title data-page-title="Mumbai Indians | MI 2025 Squad &amp; Overview | IPLT20" id="page_title">Mumbai Indians | MI 2025 Squad &amp; Overview | IPLT20</title>
<link href="https://www.iplt20.com/teams/mumbai-indians" rel="canonical"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, minimum-scale=0.5, maximum-scale=3.0" name="viewport"/>
<!-- <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no"> -->
<meta content="5JEMqZ13A5bCGMdzbhkuEkGVT1WIQDv0c2mrJWb9" name="csrf-token"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="#19398a" name="theme-color"/>
<meta content="website" name="og:type"/>
<meta content="https://www.iplt20.com/teams/mumbai-indians" name="og:url"/>
<meta content="Mumbai Indians | MI 2025 Squad &amp; Overview | IPLT20" property="og:title"/>

In [ ]:
def get_team_players(url):
    try:
        resp = requests.get(url)
        resp.raise_for_status()
        print("GOT GREEN LIGHT FROM URL!!!")
    except requests.exceptions.RequestException as e:
        print(f"Request Failed: {e}")
        return pd.DataFrame()  # Return empty DataFrame on failure

    soup = BeautifulSoup(resp.text, "html.parser")
    players = []

    

    for i in soup.find_all("a", {"data-player_name": True}):
        # Head shot png
        head_shot_tag = i.find(class_='lazyload')
        head_shot_png = head_shot_tag.get('data-src') if head_shot_tag else None

        # Player name
        player_name = i['data-player_name']

        # Player type
        player_type_tag = i.find(class_='d-block w-100 text-center')
        player_type = player_type_tag.text.strip() if player_type_tag else None

        # Scrapping team's icon
        teams_icon_div = i.find(class_='teams-icon')
        tag = teams_icon_div.find_all("img") if teams_icon_div else []

        # Scrapping individual's icons
        player_type_icon = captain_icon = foreign_player_icon = None

        if len(tag) == 1:
            player_type_icon = tag[-1]['src']
        elif len(tag) == 2:
            if "captain" in tag[0]['src']:
                captain_icon = tag[0]['src']
                player_type_icon = tag[1]['src']
            else:
                foreign_player_icon = tag[0]['src']
                player_type_icon = tag[1]['src']
        elif len(tag) >= 3:
            captain_icon = tag[0]['src']
            foreign_player_icon = tag[1]['src']
            player_type_icon = tag[2]['src']

        players.append([
            player_name,
            head_shot_png,
            player_type,
            player_type_icon,
            captain_icon,
            foreign_player_icon
        ])

    df = pd.DataFrame(players, columns=['player_name', 'head_shot_png', 'player_type','player_type_icon', 'captain_icon', 'foreign_player_icon'])
    return df


In [27]:
get_team_players(mi_url)

GOT GREEN LIGHT FROM URL!!!
MI


,player_name,head_shot_png,player_type,player_type_icon,captain_icon,foreign_player_icon
0,Rohit Sharma,https://documents.iplt20.com/ipl/IPLHeadshot20...,Batter,https://www.iplt20.com/assets/images/teams-bat...,None,None
1,Surya Kumar Yadav,https://documents.iplt20.com/ipl/IPLHeadshot20...,Batter,https://www.iplt20.com/assets/images/teams-bat...,None,None
2,Robin Minz,https://documents.iplt20.com/ipl/IPLHeadshot20...,WK-Batter,https://www.iplt20.com/assets/images/teams-wic...,None,None
3,Ryan Rickelton,https://documents.iplt20.com/ipl/IPLHeadshot20...,WK-Batter,https://www.iplt20.com/assets/images/teams-wic...,None,https://www.iplt20.com/assets/images/teams-for...
4,Shrijith Krishnan,https://documents.iplt20.com/ipl/IPLHeadshot20...,WK-Batter,https://www.iplt20.com/assets/images/teams-wic...,None,None
5,Bevon Jacobs,https://documents.iplt20.com/ipl/IPLHeadshot20...,Batter,https://www.iplt20.com/assets/images/teams-bat...,None,https://www.iplt20.com/assets/images/teams-for...
6,N. Tilak Varma,https://documents.iplt20.com/ipl/IPLHeadshot20...,Batter,https://www.iplt20.com/assets/images/teams-bat...,None,None
7,Hardik Pandya,https://documents.iplt20.com/ipl/IPLHeadshot20...,All-Rounder,https://www.iplt20.com/assets/images/teams-all...,https://www.iplt20.com/assets/images/teams-cap...,None
8,Naman Dhir,https://documents.iplt20.com/ipl/IPLHeadshot20...,All-Rounder,https://www.iplt20.com/assets/images/teams-all...,None,None
9,Will Jacks,https://documents.iplt20.com/ipl/IPLHeadshot20...,All-Rounder,https://www.iplt20.com/assets/images/teams-all...,None,https://www.iplt20.com/assets/images/teams-for...


In [28]:
get_team_players(srh_url)


GOT GREEN LIGHT FROM URL!!!
SRH


,player_name,head_shot_png,player_type,player_type_icon,captain_icon,foreign_player_icon
0,Ishan Kishan,https://documents.iplt20.com/ipl/IPLHeadshot20...,WK-Batter,https://www.iplt20.com/assets/images/teams-wic...,None,None
1,Atharva Taide,https://documents.iplt20.com/ipl/IPLHeadshot20...,Batter,https://www.iplt20.com/assets/images/teams-bat...,None,None
2,Abhinav Manohar,https://documents.iplt20.com/ipl/IPLHeadshot20...,Batter,https://www.iplt20.com/assets/images/teams-bat...,None,None
3,Aniket Verma,https://documents.iplt20.com/ipl/IPLHeadshot20...,Batter,https://www.iplt20.com/assets/images/teams-bat...,None,None
4,Sachin Baby,https://documents.iplt20.com/ipl/IPLHeadshot20...,Batter,https://www.iplt20.com/assets/images/teams-bat...,None,None
5,Smaran Ravichandran,https://documents.iplt20.com/ipl/IPLHeadshot20...,Batter,https://www.iplt20.com/assets/images/teams-bat...,None,None
6,Heinrich Klaasen,https://documents.iplt20.com/ipl/IPLHeadshot20...,WK-Batter,https://www.iplt20.com/assets/images/teams-wic...,None,https://www.iplt20.com/assets/images/teams-for...
7,Travis Head,https://documents.iplt20.com/ipl/IPLHeadshot20...,Batter,https://www.iplt20.com/assets/images/teams-bat...,None,https://www.iplt20.com/assets/images/teams-for...
8,Harshal Patel,https://documents.iplt20.com/ipl/IPLHeadshot20...,All-Rounder,https://www.iplt20.com/assets/images/teams-all...,None,None
9,Kamindu Mendis,https://documents.iplt20.com/ipl/IPLHeadshot20...,All-Rounder,https://www.iplt20.com/assets/images/teams-all...,None,https://www.iplt20.com/assets/images/teams-for...


In [ ]:
# get_team_players(mi_url).to_csv("test.csv")

GOT GREEN LIGHT FROM URL!!!


In [17]:
test1 = soup.find_all("a",{"data-player_name" : True})[5]

In [18]:
test1['data-player_name']

'Bevon Jacobs'

In [19]:
test1.find(class_ = 'd-block w-100 text-center').text

'Batter'

In [50]:
resp = requests.get(srh_url)
soup = BeautifulSoup(resp.text, "html.parser")

for i in soup.find_all("a",{"data-player_name" : True}):
    print(i['data-player_name'])
    tag = i.find(class_ = 'teams-icon').find_all("img")
    print(tag)
    if len(tag) == 1:
            player_type_icon = tag[-1]['src']
            captain_icon = None
            foreign_player_icon = None

            print(player_type_icon,captain_icon,foreign_player_icon)
    elif len(tag) == 2:
        if "captain" in tag[-2]['src']:
            player_type_icon = tag[-1]['src']
            captain_icon = tag[-2]['src']
            foreign_player_icon = None
            print(player_type_icon,captain_icon,foreign_player_icon)

        else:
            player_type_icon = tag[-1]['src']
            captain_icon = None
            foreign_player_icon = tag[-2]['src']
            print(player_type_icon,captain_icon,foreign_player_icon)

    else:
        player_type_icon = tag[-1]['src']
        foreign_player_icon = tag[-2]['src']
        captain_icon = tag[-3]['src']
        print(player_type_icon,captain_icon,foreign_player_icon)


Ishan Kishan
[<img alt="" src="https://www.iplt20.com/assets/images/teams-wicket-keeper-icon.svg"/>]
https://www.iplt20.com/assets/images/teams-wicket-keeper-icon.svg None None
Atharva Taide
[<img alt="" src="https://www.iplt20.com/assets/images/teams-batsman-icon.svg"/>]
https://www.iplt20.com/assets/images/teams-batsman-icon.svg None None
Abhinav Manohar
[<img alt="" src="https://www.iplt20.com/assets/images/teams-batsman-icon.svg"/>]
https://www.iplt20.com/assets/images/teams-batsman-icon.svg None None
Aniket Verma
[<img alt="" src="https://www.iplt20.com/assets/images/teams-batsman-icon.svg"/>]
https://www.iplt20.com/assets/images/teams-batsman-icon.svg None None
Sachin Baby
[<img alt="" src="https://www.iplt20.com/assets/images/teams-batsman-icon.svg"/>]
https://www.iplt20.com/assets/images/teams-batsman-icon.svg None None
Smaran Ravichandran
[<img alt="" src="https://www.iplt20.com/assets/images/teams-batsman-icon.svg"/>]
https://www.iplt20.com/assets/images/teams-batsman-icon.sv

In [ ]:
def get_team_players(url):
    try:
        resp = requests.get(url)
        resp.raise_for_status()
        print(f"GOT GREEN LIGHT FROM {url}!!!")
    except requests.exceptions.RequestException as e:
        print(f"Request Failed: {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(resp.text, "html.parser")
    players = []

    team_name = url.rstrip("/").split("/")[-1]

    for i in soup.find_all("a", {"data-player_name": True}):
        head_shot_tag = i.find(class_='lazyload')
        head_shot_png = head_shot_tag.get('data-src') if head_shot_tag else None

        player_name = i['data-player_name']

        player_type_tag = i.find(class_='d-block w-100 text-center')
        player_type = player_type_tag.text.strip() if player_type_tag else None

        teams_icon_div = i.find(class_='teams-icon')
        tag = teams_icon_div.find_all("img") if teams_icon_div else []

        player_type_icon = captain_icon = foreign_player_icon = None
        if len(tag) == 1:
            player_type_icon = tag[-1]['src']
        elif len(tag) == 2:
            if "captain" in tag[0]['src']:
                captain_icon = tag[0]['src']
                player_type_icon = tag[1]['src']
            else:
                foreign_player_icon = tag[0]['src']
                player_type_icon = tag[1]['src']
        elif len(tag) >= 3:
            captain_icon = tag[0]['src']
            foreign_player_icon = tag[1]['src']
            player_type_icon = tag[2]['src']

        players.append([
            player_name,
            head_shot_png,
            player_type,
            player_type_icon,
            captain_icon,
            foreign_player_icon
        ])

    df = pd.DataFrame(players, columns=[
        'player_name', 'head_shot_png', 'player_type',
        'player_type_icon', 'captain_icon', 'foreign_player_icon'
    ])
    df['team_name'] = team_name  
    return df


In [46]:
df = merge_data_for_team_players(team_urls)

GOT GREEN LIGHT FROM https://www.iplt20.com/teams/chennai-super-kings!!!
GOT GREEN LIGHT FROM https://www.iplt20.com/teams/delhi-capitals!!!
GOT GREEN LIGHT FROM https://www.iplt20.com/teams/gujarat-titans!!!
GOT GREEN LIGHT FROM https://www.iplt20.com/teams/kolkata-knight-riders!!!
GOT GREEN LIGHT FROM https://www.iplt20.com/teams/lucknow-super-giants!!!
GOT GREEN LIGHT FROM https://www.iplt20.com/teams/mumbai-indians!!!
GOT GREEN LIGHT FROM https://www.iplt20.com/teams/punjab-kings!!!
GOT GREEN LIGHT FROM https://www.iplt20.com/teams/rajasthan-royals!!!
GOT GREEN LIGHT FROM https://www.iplt20.com/teams/royal-challengers-bengaluru!!!
GOT GREEN LIGHT FROM https://www.iplt20.com/teams/sunrisers-hyderabad!!!


In [47]:
df['team_name'].value_counts()

team_name
chennai-super-kings            25
gujarat-titans                 25
punjab-kings                   25
lucknow-super-giants           24
mumbai-indians                 23
delhi-capitals                 22
royal-challengers-bengaluru    22
kolkata-knight-riders          21
rajasthan-royals               20
sunrisers-hyderabad            20
Name: count, dtype: int64

In [48]:
df

,player_name,head_shot_png,player_type,player_type_icon,captain_icon,foreign_player_icon,team_name
0,MS Dhoni,https://documents.iplt20.com/ipl/IPLHeadshot20...,WK-Batter,https://www.iplt20.com/assets/images/teams-wic...,https://www.iplt20.com/assets/images/teams-cap...,None,chennai-super-kings
1,Dewald Brevis,https://documents.iplt20.com/ipl/IPLHeadshot20...,Batter,https://www.iplt20.com/assets/images/teams-bat...,None,https://www.iplt20.com/assets/images/teams-for...,chennai-super-kings
2,Devon Conway,https://documents.iplt20.com/ipl/IPLHeadshot20...,Batter,https://www.iplt20.com/assets/images/teams-bat...,None,https://www.iplt20.com/assets/images/teams-for...,chennai-super-kings
3,Rahul Tripathi,https://documents.iplt20.com/ipl/IPLHeadshot20...,Batter,https://www.iplt20.com/assets/images/teams-bat...,None,None,chennai-super-kings
4,Shaik Rasheed,https://documents.iplt20.com/ipl/IPLHeadshot20...,Batter,https://www.iplt20.com/assets/images/teams-bat...,None,None,chennai-super-kings
...,...,...,...,...,...,...,...
222,Rahul Chahar,https://documents.iplt20.com/ipl/IPLHeadshot20...,Bowler,https://www.iplt20.com/assets/images/teams-bow...,None,None,sunrisers-hyderabad
223,Simarjeet Singh,https://documents.iplt20.com/ipl/IPLHeadshot20...,Bowler,https://www.iplt20.com/assets/images/teams-bow...,None,None,sunrisers-hyderabad
224,Zeeshan Ansari,https://documents.iplt20.com/ipl/IPLHeadshot20...,Bowler,https://www.iplt20.com/assets/images/teams-bow...,None,None,sunrisers-hyderabad
225,Jaydev Unadkat,https://documents.iplt20.com/ipl/IPLHeadshot20...,Bowler,https://www.iplt20.com/assets/images/teams-bow...,None,None,sunrisers-hyderabad


In [ ]:
df = pd.read_csv("scrapped_teams_info/ipl_teams_data.csv")


In [15]:
df

,team_name,abbrevation,trophy_year,logo,team_url
0,Chennai Super Kings,CSK,2010 | 2011 | 2018 | 2021 | 2023,https://documents.iplt20.com/ipl/CSK/logos/Log...,https://www.iplt20.com/teams/chennai-super-kings
1,Delhi Capitals,DC,NaN,https://documents.iplt20.com/ipl/DC/Logos/Logo...,https://www.iplt20.com/teams/delhi-capitals
2,Gujarat Titans,GT,2022,https://documents.iplt20.com/ipl/GT/Logos/Logo...,https://www.iplt20.com/teams/gujarat-titans
3,Kolkata Knight Riders,KKR,2012 | 2014 | 2024,https://documents.iplt20.com/ipl/KKR/Logos/Log...,https://www.iplt20.com/teams/kolkata-knight-ri...
4,Lucknow Super Giants,LSG,NaN,https://documents.iplt20.com/ipl/LSG/Logos/Log...,https://www.iplt20.com/teams/lucknow-super-giants
5,Mumbai Indians,MI,2013 | 2015 | 2017 | 2019 | 2020,https://documents.iplt20.com/ipl/MI/Logos/Logo...,https://www.iplt20.com/teams/mumbai-indians
6,Punjab Kings,PBKS,NaN,https://documents.iplt20.com/ipl/PBKS/Logos/Lo...,https://www.iplt20.com/teams/punjab-kings
7,Rajasthan Royals,RR,2008,https://documents.iplt20.com/ipl/RR/Logos/Logo...,https://www.iplt20.com/teams/rajasthan-royals
8,Royal Challengers Bengaluru,RCB,NaN,https://documents.iplt20.com/ipl/RCB/Logos/Log...,https://www.iplt20.com/teams/royal-challengers...
9,Sunrisers Hyderabad,SRH,2016,https://documents.iplt20.com/ipl/SRH/Logos/Log...,https://www.iplt20.com/teams/sunrisers-hyderabad


In [50]:
resp = requests.get("https://www.iplt20.com/match/2025/1839")

In [53]:
soup = BeautifulSoup(resp.text)

In [74]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()

driver.get("https://www.iplt20.com/match/2025/1839")
driver.find_elements(By.CLASS_NAME, "cmdEvent fourCmd mcBall mcBall mcBall mcBall mcBall mcBall")
time.sleep(10)
driver.close()

In [77]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("http://www.python.org")
assert "Python" in driver.title
elem = driver.find_element(By.NAME, "q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source
driver.close()

In [3]:
from bs4 import BeautifulSoup

In [3]:
kkr = driver.page_source

In [4]:
soup = BeautifulSoup(driver.page_source)

In [5]:
soup

<html lang="en"><head><style type="text/css">@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}.ng-animate-shim{visibility:hidden;}.ng-anchor{position:absolute;}</style><style data-vjs-version="7.21.4" id="bc-style-vjs" type="text/css">@charset "UTF-8";.video-js .vjs-big-play-button .vjs-icon-placeholder:before,.video-js .vjs-modal-dialog,.vjs-button>.vjs-icon-placeholder:before,.vjs-modal-dialog .vjs-modal-dialog-content{position:absolute;top:0;left:0;width:100%;height:100%}.video-js .vjs-big-play-button .vjs-icon-placeholder:before,.vjs-button>.vjs-icon-placeholder:before{text-align:center}@font-face{font-family:VideoJS;src:url(data:application/font-woff;charset=utf-8;base64,d09GRgABAAAAABDkAAsAAAAAG6gAAQAAAAAAAAAAAAAAAAAAAAAAAAAAAABHU1VCAAABCAAAADsAAABUIIslek9TLzIAAAFEAAAAPgAAAFZRiV3hY21hcAAAAYQAAADaAAADPv749/pnbHlmAAACYAAAC3AAABHQZg6OcWhlYWQAAA3QAAAAKwAAADYZw251a

In [14]:
soup.find(class_="matchOrder mob-hide ng-binding ng-scope").text

'Match 1 '

In [15]:
soup.find(class_="ms-matchdate ng-binding").text

'22 Mar 2025'

In [16]:
soup.find(class_="matGround ng-binding").text

'Eden Gardens, Kolkata'

In [19]:
soup.find_all(class_="cmdOver mcBall bgFour mcBall mcBall mcBall mcBall mcBall")[0].text

' 16.2 4       '

[<div class="ppText ng-binding ng-scope" ng-bind-html="matchSummary.PostMatchCommentary | to_trusted" ng-if="matchSummary.PostMatchCommentary != undefined &amp;&amp; matchSummary.PostMatchCommentary !=''"><p><strong>Powerplay (Overs 1-6)</strong><br/>
 <span style="color:#8e44ad"><strong>KKR: 60/1 (10 rpo, 8x4, 4x6)</strong></span><strong>  ll  <span style="color:#c0392b">RCB: 80/0 (13.33 rpo, 10x4, 4x6)</span></strong><br/>
 <br/>
 <strong>Middle Overs (Overs 7-15)</strong><br/>
 <span style="color:#8e44ad"><strong>KKR: 85/4 (9.44 rpo, 8x4, 3x6)</strong></span><strong>  ll  <span style="color:#c0392b">RCB: 77/2 (8.55 rpo, 8x4, 2x6)</span></strong><br/>
 <br/>
 <strong>Death Overs (Overs 16-20)</strong><br/>
 <span style="color:#8e44ad"><strong>KKR: 29/3 (5.80 rpo, 2x6, 1x6) </strong></span><strong>ll  <span style="color:#c0392b">RCB: 20/1 in 1.2 overs (15 rpo, 3x4, 1x6)</span></strong><br/>
 <br/>
  </p>
 <p><strong>Royal Challengers Bengaluru (RCB) kicked off their IPL 2025 campaign 

In [44]:
print(ball_wrapper[1].find(class_ = "commentaryText ng-binding").text)

RCB SEAL THE DEAL WITH AUTHORITY full length ball, pitching outside off stump, Liam Livingstone swings across the line, and even with the inner half of the bat, he gets enough on it. Beats mid-on and it races away! 
 


In [ ]:
# commentary
ball_wrapper[0].find(class_ = "commentaryText ng-binding").text

'RCB SEAL THE DEAL WITH AUTHORITY full length ball, pitching outside off stump, Liam Livingstone swings across the line, and even with the inner half of the bat, he gets enough on it. Beats mid-on and it races away!\xa0\n '

In [50]:
ball_wrapper[0]

<div class="ballWrapper ng-scope" ng-repeat="overHistoryData in list"> <!-- ngRepeat: nblist in newBatsmanList --><!-- ngIf: overHistoryData.IsWicket == '1' && nblist.OverNo == overHistoryData.OverNo && nblist.ActualBallNo == overHistoryData.ActualBallNo && nblist.InningsNo == overHistoryData.InningsNo && IsMatchEnd == 0 --><!-- end ngRepeat: nblist in newBatsmanList --><!-- ngIf: overHistoryData.IsWicket == '1' && nblist.OverNo == overHistoryData.OverNo && nblist.ActualBallNo == overHistoryData.ActualBallNo && nblist.InningsNo == overHistoryData.InningsNo && IsMatchEnd == 0 --><!-- end ngRepeat: nblist in newBatsmanList --><!-- ngIf: overHistoryData.IsWicket == '1' && nblist.OverNo == overHistoryData.OverNo && nblist.ActualBallNo == overHistoryData.ActualBallNo && nblist.InningsNo == overHistoryData.InningsNo && IsMatchEnd == 0 --><!-- end ngRepeat: nblist in newBatsmanList --> <div class="cmdEvent fourCmd mcBall bgFour mcBall mcBall mcBall mcBall mcBall" ng-class="{'cmdEventWithImage

In [ ]:
ball_wrapper = soup.find_all(class_ = "ballWrapper ng-scope")[1:]
l = []
for i in ball_wrapper:
    commentary_tag = i.find(class_="commentaryText ng-binding")
    commentry = commentary_tag.text if commentary_tag else None

    over_tag = i.find("p", class_="cmdOver")
    over_and_runs = over_tag.text.strip() if over_tag else None

    l.append([commentry,over_and_runs])

    import pandas as pd

# Sample DataFrame
df = pd.DataFrame(l, columns=['commentary', 'over_and_run'])

# Drop rows where 'over_and_run' is NaN
df = df[df['over_and_run'].notna()]

# Split the string and ensure it's a list
df['over_and_run'] = df['over_and_run'].apply(lambda x: x.split() if isinstance(x, str) else [])

# Now extract elements safely
df['over'] = df['over_and_run'].apply(lambda x: x[0] if len(x) > 0 else None)
df['run'] = df['over_and_run'].apply(lambda x: x[1] if len(x) > 1 else None)
df['extras'] = df['over_and_run'].apply(lambda x: x[2] if len(x) > 2 else None)



In [25]:
import pandas as pd
df = pd.DataFrame(l, columns = ['commentary','over_and_run'])

# Seperate out the over, runs and wickets
df = df[df['over_and_run'].notna()]
df["over_and_run"] = df['over_and_run'].apply(lambda x : x.split(' '))

In [26]:
df

,commentary,over_and_run
0,RCB SEAL THE DEAL WITH AUTHORITY full length b...,"[16.2, 4]"
1,"BANG! short length ball, pitching outside leg...","[16.1, 6]"
3,"Short of good length ball, pitching outside of...","[15.6, 1]"
4,"Good length ball, pitching outside off stump, ...","[15.5, 0]"
5,"STARTS OFF WITH A FOUR short length ball, pit...","[15.4, 4]"
...,...,...
96,"Full length ball, pitching on middle stump, Ph...","[0.4, 1]"
97,"Good length ball, pitching outside off stump, ...","[0.3, 0]"
98,"Good length ball, pitching outside off stump, ...","[0.2, 0]"
99,"Full length ball, pitching on leg stump, Phil ...","[0.2, , 1, WD]"


In [29]:
df['run'] = df['over_and_run'].apply(lambda x : x[1])

In [41]:
df[df['run'] != 'W']['run'].astype("int").sum()

177

In [42]:
df[df['run'] == 'W'].shape[0]

3

In [44]:
df['extras'].unique()

array([None, 'LB', 'WD'], dtype=object)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   commentary    99 non-null     object
 1   over_and_run  99 non-null     object
dtypes: object(2)
memory usage: 1.7+ KB


In [64]:
ball_wrapper[0].find("p", class_="cmdOver mcBall bgFour mcBall mcBall mcBall mcBall mcBall").text

' 16.2 4       '

In [47]:
tabs = driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')

In [48]:
tabs

[<selenium.webdriver.remote.webelement.WebElement (session="01928ccadf8ac2eff6525f44305cedc3", element="f.C42B8CE73328EAD7482AF9851E96ABBF.d.9A2BAAF7BB00F52F491EEFF407423375.e.92")>,
 <selenium.webdriver.remote.webelement.WebElement (session="01928ccadf8ac2eff6525f44305cedc3", element="f.C42B8CE73328EAD7482AF9851E96ABBF.d.9A2BAAF7BB00F52F491EEFF407423375.e.108")>,
 <selenium.webdriver.remote.webelement.WebElement (session="01928ccadf8ac2eff6525f44305cedc3", element="f.C42B8CE73328EAD7482AF9851E96ABBF.d.9A2BAAF7BB00F52F491EEFF407423375.e.111")>,
 <selenium.webdriver.remote.webelement.WebElement (session="01928ccadf8ac2eff6525f44305cedc3", element="f.C42B8CE73328EAD7482AF9851E96ABBF.d.9A2BAAF7BB00F52F491EEFF407423375.e.95")>,
 <selenium.webdriver.remote.webelement.WebElement (session="01928ccadf8ac2eff6525f44305cedc3", element="f.C42B8CE73328EAD7482AF9851E96ABBF.d.9A2BAAF7BB00F52F491EEFF407423375.e.144")>,
 <selenium.webdriver.remote.webelement.WebElement (session="01928ccadf8ac2eff6525f

In [49]:
for tab in tabs:
    print(tab.text)

COMMENTARY
SCORECARD
TEAMS
VIDEOS

PHOTOS
WAGON WHEEL
MANHATTAN & WORM



KKR Innings
RCB Innings
























In [70]:
for tab in tabs:
    if "KKR" in tab.text:
        tab.click()
        time.sleep(5)
        


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00F3D363+60275]
	GetHandleVerifier [0x00F3D3A4+60340]
	(No symbol) [0x00D706F3]
	(No symbol) [0x00D76E47]
	(No symbol) [0x00D7913A]
	(No symbol) [0x00D791B7]
	(No symbol) [0x00DB33AE]
	(No symbol) [0x00DDD08C]
	(No symbol) [0x00DADCF4]
	(No symbol) [0x00DDD304]
	(No symbol) [0x00DFE6EB]
	(No symbol) [0x00DDCE86]
	(No symbol) [0x00DAC623]
	(No symbol) [0x00DAD474]
	GetHandleVerifier [0x01188FE3+2467827]
	GetHandleVerifier [0x011845E6+2448886]
	GetHandleVerifier [0x0119F80C+2560028]
	GetHandleVerifier [0x00F53DF5+153093]
	GetHandleVerifier [0x00F5A3BD+179149]
	GetHandleVerifier [0x00F44BB8+91080]
	GetHandleVerifier [0x00F44D60+91504]
	GetHandleVerifier [0

In [55]:
kkr_innings = driver.page_source
kkr_soup = BeautifulSoup(kkr_innings)

In [58]:
ball_wrapper = kkr_soup.find_all(class_ = "ballWrapper ng-scope")[1:]
l = []
for i in ball_wrapper:
    commentary_tag = i.find(class_="commentaryText ng-binding")
    commentry = commentary_tag.text if commentary_tag else None

    over_tag = i.find("p", class_="cmdOver")
    over_and_runs = over_tag.text.strip() if over_tag else None

    l.append([commentry,over_and_runs])

    import pandas as pd

# Sample DataFrame
df = pd.DataFrame(l, columns=['commentary', 'over_and_run'])

# Drop rows where 'over_and_run' is NaN
df = df[df['over_and_run'].notna()]

# Split the string and ensure it's a list
df['over_and_run'] = df['over_and_run'].apply(lambda x: x.split() if isinstance(x, str) else [])

# Now extract elements safely
df['over'] = df['over_and_run'].apply(lambda x: x[0] if len(x) > 0 else None)
df['run'] = df['over_and_run'].apply(lambda x: x[1] if len(x) > 1 else None)
df['extras'] = df['over_and_run'].apply(lambda x: x[2] if len(x) > 2 else None)



In [59]:
df

,commentary,over_and_run,over,run,extras
0,"Short of good length ball, pitching outside le...","[19.6, 1]",19.6,1,None
1,UP IN THE AIR...... AND TAKES IT short length...,"[19.5, W]",19.5,W,None
2,"FORTUNE FAVOURS THE BRAVE short length ball,...","[19.4, 4]",19.4,4,None
3,"Short length ball, pitching outside off stump,...","[19.3, 0]",19.3,0,None
4,"Short length ball, pitching outside off stump,...","[19.2, 0]",19.2,0,None
...,...,...,...,...,...
120,"EDGED AND GONE! short of good length ball, pi...","[0.5, W]",0.5,W,None
121,"Short of good length ball, pitching on leg stu...","[0.4, 0]",0.4,0,None
122,"Short of good length ball, pitching outside le...","[0.3, 0]",0.3,0,None
123,BOUNDARY NUMBER ONE IN THE IPL 2025 short of ...,"[0.2, 4]",0.2,4,None


In [61]:
df['extras'].value_counts()

extras
WD    4
LB    2
Name: count, dtype: int64

In [69]:
str(df[df['run'] != 'W']['run'].astype("int").sum()) + '/' +str( df[df['run'] == 'W']['run'].shape[0])

'174/8'

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
driver = webdriver.Chrome()


url = "https://www.iplt20.com/match/2025/1799"
driver.get(url)

In [6]:
driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')


[<selenium.webdriver.remote.webelement.WebElement (session="eeeb3de012792ea817f12891e9f1fcce", element="f.F187A8576DCAAA9BA9204B3C73E046EA.d.8B421A67D2DFA1E60F452F94A7790384.e.98")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eeeb3de012792ea817f12891e9f1fcce", element="f.F187A8576DCAAA9BA9204B3C73E046EA.d.8B421A67D2DFA1E60F452F94A7790384.e.99")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eeeb3de012792ea817f12891e9f1fcce", element="f.F187A8576DCAAA9BA9204B3C73E046EA.d.8B421A67D2DFA1E60F452F94A7790384.e.100")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eeeb3de012792ea817f12891e9f1fcce", element="f.F187A8576DCAAA9BA9204B3C73E046EA.d.8B421A67D2DFA1E60F452F94A7790384.e.82")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eeeb3de012792ea817f12891e9f1fcce", element="f.F187A8576DCAAA9BA9204B3C73E046EA.d.8B421A67D2DFA1E60F452F94A7790384.e.813")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eeeb3de012792ea817f1289

In [9]:
for i in driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]'):
    print(i.text)

COMMENTARY
SCORECARD
TEAMS
VIDEOS

PHOTOS
WAGON WHEEL
MANHATTAN & WORM



KKR Innings
RCB Innings
























In [10]:
driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')[11].text

'KKR Innings'

In [14]:
driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')[11].click()

In [16]:
driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')[11]
first_inning_soup = driver.page_source


In [19]:
from bs4 import BeautifulSoup

In [20]:
first_innging = BeautifulSoup(first_inning_soup)

In [15]:
driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')[12].click()

In [ ]:
first_innging.find_all(class_ = 'ap-inner-tb-click ng-binding ng-scope')[0].text

'KKR Innings'

In [30]:
driver.quit()

KKR Innings
RCB Innings


In [39]:
for i in first_soup.find_all(class_ = 'ap-inner-tb-click ng-binding ng-scope'):
    print(i.text)


RCB Innings  
RCB Innings  


In [40]:
for i in second_soup.find_all(class_ = 'ap-inner-tb-click ng-binding ng-scope'):
    print(i.text)


KKR Innings  
KKR Innings  


In [41]:
driver = webdriver.Chrome()
driver.get(url)
tag_to_switch_innings = driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')
first_inning = tag_to_switch_innings[11]

In [44]:
tag_to_switch_innings[11].text

'KKR Innings'

In [57]:
ball_wrapper = first_soup.find_all(class_ = "ballWrapper ng-scope")[1:]
l = []
for i in ball_wrapper:
    commentary_tag = i.find(class_="commentaryText ng-binding")
    commentry = commentary_tag.text if commentary_tag else None

    over_tag = i.find("p", class_="cmdOver")
    over_and_runs = over_tag.text.strip() if over_tag else None

    l.append([commentry,over_and_runs])

    import pandas as pd

# Sample DataFrame
df = pd.DataFrame(l, columns=['commentary', 'over_and_run'])

# Drop rows where 'over_and_run' is NaN
df = df[df['over_and_run'].notna()]

# Split the string and ensure it's a list
df['over_and_run'] = df['over_and_run'].apply(lambda x: x.split() if isinstance(x, str) else [])

# Now extract elements safely
df['over'] = df['over_and_run'].apply(lambda x: x[0] if len(x) > 0 else None)
df['run'] = df['over_and_run'].apply(lambda x: x[1] if len(x) > 1 else None)
df['extras'] = df['over_and_run'].apply(lambda x: x[2] if len(x) > 2 else None)



In [59]:
driver.quit()

In [50]:
for i in kkr_soup.find_all(class_ = 'ap-inner-tb-click ng-binding ng-scope'):
    print(i.text)

RCB Innings  
RCB Innings  


In [51]:
kkr_soup.find_all(class_ = 'ap-inner-tb-click ng-binding ng-scope')

[<a class="ap-inner-tb-click ng-binding ng-scope" href="javascript:void(0);" ng-class="{'ap-active-team' : currentInnText == 'Innings'+list.innNo}" ng-click="inningsTabsChange(matchSummary.MatchID,'dropdownInnChange','innchange',list.innNo)" ng-if="list.innNo &lt;= 2" ng-repeat="list in inningsList">RCB <span class="mob-hide">Innings </span> </a>,
 <a class="ap-inner-tb-click ng-binding ng-scope" href="javascript:void(0);" ng-class="{'ap-active-team' : currentInnText == 'Innings'+list.innNo}" ng-click="inningsTabsChange(matchSummary.MatchID,'dropdownInnChange','innchange',list.innNo)" ng-if="list.innNo &lt;= 2" ng-repeat="list in inningsList">RCB <span class="mob-hide">Innings </span> </a>]

In [4]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

import pandas as pd

In [5]:
def get_match_soups(url):
    
    driver = webdriver.Chrome()
    driver.get(url)
    tag_to_switch_innings = driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')
    first_inning = tag_to_switch_innings[11]
    whos_first_inning = first_inning.text
    first_inning.click()
    time.sleep(3)
    first_inning_soup = BeautifulSoup(driver.page_source, "html.parser")
    second_inning = tag_to_switch_innings[12]
    whos_second_inning = second_inning.text
    second_inning.click()
    time.sleep(3)
    second_inning_soup = BeautifulSoup(driver.page_source, "html.parser")

    driver.quit()

    return first_inning_soup,second_inning_soup, whos_first_inning, whos_second_inning

In [6]:
first_inning_soup,second_inning_soup, whos_first_inning, whos_second_inning = get_match_soups("https://www.iplt20.com/match/2025/1799")

In [7]:
def get_match_metadata(soup):
    """
    The function provides metadata of the match up between two teams.
    Input : Provide the soup of the innings data
    Ouput : Gives out a array of [Match Number, Venue, Date, Time]
    """
    l = []
    match_number = soup.find(class_ = "matchOrder mob-hide ng-binding ng-scope").text 
    l.append(match_number)
    
    for i in soup.find(class_ = "ap-match-place col-100 floatLft textCenter re ng-scope").find_all('span'):
        l.append(i.text)

    return l

In [8]:
get_match_metadata(first_inning_soup)

['Match 1 ', 'Eden Gardens, Kolkata', '22 Mar 2025', '7:30 pm IST ']

In [ ]:
def get_ball_by_ball_commentary(soup):
    # ball_wrapper = soup.find_all('div', class_ = ['ballWrapper ng-scope','cmdOver mcBall mcBall mcBall mcBall mcBall mcBall'])
    ball_wrapper = soup.select('div.ballWrapper.ng-scope')
    ball_wrapper = ball_wrapper[1:]
    
    # To Store the scrapped data
    l = []

    for i in ball_wrapper:
        if i.p.contents[0].strip() != '':
            ball_number = i.p.contents[0].strip()
            event_on_that_ball = i.find('i').text.strip()

            # Extract bowler vs batter info
            bowling_info_tag = i.find('div', class_='commentaryStartText')
            bowling_info = bowling_info_tag.get_text(strip=True) if bowling_info_tag else None

            # Extract detailed commentary
            commentary_tag = i.find('div', class_='commentaryText')
            commentary = commentary_tag.get_text(strip=True) if commentary_tag else None

            l.append({
                'ball': ball_number,
                'event': event_on_that_ball,
                'bowling_info': bowling_info,
                'commentary': commentary,
            })

    df = pd.DataFrame(l)
    return df
    

    

In [12]:
get_ball_by_ball_commentary(second_inning_soup)

,ball,event,bowling_info,commentary,Innings
0,16.2,4,Spencer Johnson bowling to Liam Livingstone,RCB SEAL THE DEAL WITH AUTHORITYfull length ba...,RCB
1,16.1,6,Spencer Johnson bowling to Liam Livingstone,"BANG! short length ball, pitching outside leg...",RCB
2,15.6,1,Vaibhav Arora bowling to Liam Livingstone,"Short of good length ball, pitching outside of...",RCB
3,15.5,0,Vaibhav Arora bowling to Liam Livingstone,"Good length ball, pitching outside off stump, ...",RCB
4,15.4,4,Vaibhav Arora bowling to Liam Livingstone,"STARTS OFF WITH A FOURshort length ball, pitch...",RCB
...,...,...,...,...,...
94,0.4,1,Vaibhav Arora bowling to Phil Salt,"Full length ball, pitching on middle stump, Ph...",RCB
95,0.3,0,Vaibhav Arora bowling to Phil Salt,"Good length ball, pitching outside off stump, ...",RCB
96,0.2,0,Vaibhav Arora bowling to Phil Salt,"Good length ball, pitching outside off stump, ...",RCB
97,0.2,1 WD,Vaibhav Arora bowling to Phil Salt,"Full length ball, pitching on leg stump, Phil ...",RCB


In [ ]:
l = []
for i in get_ball_by_ball_commentary(second_inning_soup):
    if i.p.contents[0].strip() != '':
        ball_number = i.p.contents[0]
        event_on_that_ball = i.find('i').text
        

In [ ]:
l = []

for i in get_ball_by_ball_commentary(second_inning_soup):
    if i.p.contents[0].strip() != '':
        ball_number = i.p.contents[0].strip()
        event_on_that_ball = i.find('i').text.strip()

        # Extract bowler vs batter info
        bowling_info_tag = i.find('div', class_='commentaryStartText')
        bowling_info = bowling_info_tag.get_text(strip=True) if bowling_info_tag else None

        # Extract detailed commentary
        commentary_tag = i.find('div', class_='commentaryText')
        commentary = commentary_tag.get_text(strip=True) if commentary_tag else None

        l.append({
            'ball': ball_number,
            'event': event_on_that_ball,
            'bowling_info': bowling_info,
            'commentary': commentary
            'Innings' : 
        })


In [100]:
df = pd.DataFrame(l)

In [113]:
df

,ball,event,bowling_info,commentary
0,16.2,4,Spencer Johnson bowling to Liam Livingstone,RCB SEAL THE DEAL WITH AUTHORITYfull length ba...
1,16.1,6,Spencer Johnson bowling to Liam Livingstone,"BANG! short length ball, pitching outside leg..."
2,15.6,1,Vaibhav Arora bowling to Liam Livingstone,"Short of good length ball, pitching outside of..."
3,15.5,0,Vaibhav Arora bowling to Liam Livingstone,"Good length ball, pitching outside off stump, ..."
4,15.4,4,Vaibhav Arora bowling to Liam Livingstone,"STARTS OFF WITH A FOURshort length ball, pitch..."
...,...,...,...,...
94,0.4,1,Vaibhav Arora bowling to Phil Salt,"Full length ball, pitching on middle stump, Ph..."
95,0.3,0,Vaibhav Arora bowling to Phil Salt,"Good length ball, pitching outside off stump, ..."
96,0.2,0,Vaibhav Arora bowling to Phil Salt,"Good length ball, pitching outside off stump, ..."
97,0.2,1 WD,Vaibhav Arora bowling to Phil Salt,"Full length ball, pitching on leg stump, Phil ..."


In [102]:
df['event'].value_counts()

event
1       38
0       22
4       21
6        7
2        5
W        3
1 LB     2
1 WD     1
Name: count, dtype: int64

In [106]:
df[df['ball'] == '15.3']

,ball,event,bowling_info,commentary
5,15.3,W,Vaibhav Arora bowling to Rajat Patidar,"FUN ENDS FOR THE SKIPgood length ball, pitchin..."


In [111]:
print(df[df['ball'] == '15.3']['commentary'].iloc[0])

FUN ENDS FOR THE SKIPgood length ball, pitching outside off stump, Rajat Patidar looking to finish things quickly, he ends up mistiming one straight into the hands of Rinku Singh at deep mid-wicket!WICKET Rajat Patidar c Rinku Singh b Vaibhav Arora (Royal Challengers Bengaluru 162 Runs for 3 wickets)


In [112]:
print(df[df['ball'] == '11.4']['commentary'].iloc[0])


NARINE STRIKESfull length ball, pitching outside off stump, Devdutt Padikkal dances down the track looking to go big across the line — but he doesn't quite get the timing or power. It skies towards the deep and safely into the hands of the fielder.WICKET Devdutt Padikkal c Ramandeep Singh b Sunil Narine (Royal Challengers Bengaluru 118 Runs for 2 wickets)


In [20]:
get_ball_by_ball_commentary(first_inning_soup)[0].select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n")

['Josh Hazlewood bowling to Spencer Johnson',
 'Short of good length ball, pitching outside leg stump, Spencer Johnson uncomfortably plays an aggressive pull shot on the back foot for a single']

In [24]:
len(get_ball_by_ball_commentary(first_inning_soup)[0])

21

In [40]:
get_ball_by_ball_commentary(first_inning_soup)[10].text

'    18.2 1         Yash Dayal bowling to Angkrish Raghuvanshi Short length ball, pitching outside leg stump, Angkrish Raghuvanshi plays an aggressive pull shot on the back foot for a single towards Deep Mid-Wicket       '

In [43]:
len(get_ball_by_ball_commentary(second_inning_soup))

101

In [56]:
get_ball_by_ball_commentary(second_inning_soup)[1].text


'    16.1 6         Spencer Johnson bowling to Liam Livingstone BANG!\xa0 short length ball, pitching outside leg stump, Liam Livingstone goes back, gets inside the line, and hooks it flat and hard and the ball goes beyond the fine leg ropes for a maximum       '

In [54]:
for i in get_ball_by_ball_commentary(second_inning_soup):
    print(i.text)


    16.2 4         Spencer Johnson bowling to Liam Livingstone RCB SEAL THE DEAL WITH AUTHORITY full length ball, pitching outside off stump, Liam Livingstone swings across the line, and even with the inner half of the bat, he gets enough on it. Beats mid-on and it races away! 
        
    16.1 6         Spencer Johnson bowling to Liam Livingstone BANG!  short length ball, pitching outside leg stump, Liam Livingstone goes back, gets inside the line, and hooks it flat and hard and the ball goes beyond the fine leg ropes for a maximum       
                    Enjoyed your captain's innings? Rajat Patidar sprinkled his elegant touch to the chase with a quick-fire 34(16) ????@RCBTweets moving closer to the target ????Updates ? https://t.co/C9xIFpQDTn#TATAIPL | #KKRvRCB | @RCBTweets | @rrjjt_01 pic.twitter.com/1P7buQ8m0O— IndianPremierLeague (@IPL) March 22, 2025       
    15.6 1         Vaibhav Arora bowling to Liam Livingstone Short of good length ball, pitching outside off stump, Lia

In [23]:
for i in get_ball_by_ball_commentary(first_inning_soup):
    i.select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n")


AttributeError: 'NoneType' object has no attribute 'get_text'

In [21]:
get_ball_by_ball_commentary(first_inning_soup)[0].select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n")


['Josh Hazlewood bowling to Spencer Johnson',
 'Short of good length ball, pitching outside leg stump, Spencer Johnson uncomfortably plays an aggressive pull shot on the back foot for a single']

In [19]:
l = []
for i in get_ball_by_ball_commentary(first_inning_soup):
    l.append(i.select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n"))


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


AttributeError: 'NoneType' object has no attribute 'get_text'

In [73]:
get_ball_by_ball_commentary(first_inning_soup)[0].select_one("div.cmdText div.commentaryText")

<div class="commentaryText ng-binding" ng-bind-html="overHistoryData.UPDCommentry | to_trusted">Short of good length ball, pitching outside leg stump, Spencer Johnson uncomfortably plays an aggressive pull shot on the back foot for a single<p></p>
<div class="twitter-tweet twitter-tweet-rendered" style="display: flex; max-width: 550px; width: 100%; margin-top: 10px; margin-bottom: 10px;"><iframe allowfullscreen="true" allowtransparency="true" class="" data-tweet-id="1903473533059277107" frameborder="0" id="twitter-widget-0" scrolling="no" src="https://platform.twitter.com/embed/Tweet.html?dnt=false&amp;embedId=twitter-widget-0&amp;features=eyJ0ZndfdGltZWxpbmVfbGlzdCI6eyJidWNrZXQiOltdLCJ2ZXJzaW9uIjpudWxsfSwidGZ3X2ZvbGxvd2VyX2NvdW50X3N1bnNldCI6eyJidWNrZXQiOnRydWUsInZlcnNpb24iOm51bGx9LCJ0ZndfdHdlZXRfZWRpdF9iYWNrZW5kIjp7ImJ1Y2tldCI6Im9uIiwidmVyc2lvbiI6bnVsbH0sInRmd19yZWZzcmNfc2Vzc2lvbiI6eyJidWNrZXQiOiJvbiIsInZlcnNpb24iOm51bGx9LCJ0ZndfZm9zbnJfc29mdF9pbnRlcnZlbnRpb25zX2VuYWJsZWQiOnsiYnVja2V0

In [74]:
commentary_div = get_ball_by_ball_commentary(first_inning_soup)[0].select_one("div.cmdText div.commentaryText")
commentary_text = commentary_div.get_text(strip=True)
print(commentary_text)


Short of good length ball, pitching outside leg stump, Spencer Johnson uncomfortably plays an aggressive pull shot on the back foot for a singleInnings Break!#RCBwith a strong comeback after#KKRstarted well ????????Who is winning the season opener - ???? or ??Chase on the other side ??Scorecard ?https://t.co/C9xIFpQDTn#TATAIPL|#KKRvRCB|@KKRiders|@RCBTweetspic.twitter.com/mu4Ws78ddA— IndianPremierLeague (@IPL)March 22, 2025


In [81]:
get_ball_by_ball_commentary(first_inning_soup)[0].select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n")

['Josh Hazlewood bowling to Spencer Johnson',
 'Short of good length ball, pitching outside leg stump, Spencer Johnson uncomfortably plays an aggressive pull shot on the back foot for a single',
 'Innings Break!',
 '#RCB',
 'with a strong comeback after',
 '#KKR',
 'started well ????????',
 'Who is winning the season opener - ???? or ??',
 'Chase on the other side ??',
 'Scorecard ?',
 'https://t.co/C9xIFpQDTn',
 '#TATAIPL',
 '|',
 '#KKRvRCB',
 '|',
 '@KKRiders',
 '|',
 '@RCBTweets',
 'pic.twitter.com/mu4Ws78ddA',
 '— IndianPremierLeague (@IPL)',
 'March 22, 2025']

In [82]:
get_ball_by_ball_commentary(first_inning_soup)[1].select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n")


['Josh Hazlewood bowling to Harshit Rana',
 'UP IN THE AIR...... AND TAKES IT',
 'short length ball, pitching outside off stump, this time Rana doesn’t get enough of it. Comes right off the splice and balloons high into the Kolkata sky!Jitesh tracks it beautifully, running in from behind the stumps towards square leg — eyes on the ball all the way, and takes it cleanly with both hands.',
 'WICKET Harshit Rana c Jitesh Sharma b Josh Hazlewood (Kolkata Knight Riders 173 Runs for 8 wickets)']

In [83]:
get_ball_by_ball_commentary(first_inning_soup)[2].select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n")


['Josh Hazlewood bowling to Harshit Rana',
 'FORTUNE FAVOURS THE BRAVE',
 "short length ball, pitching outside off stump, Harshit Rana throws everything at it, absolutely no half-measures. Gets a big top edge, and it sails right over the keeper's head to the boundary."]

In [87]:
test = get_ball_by_ball_commentary(first_inning_soup)[-5].select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n")


In [91]:
for i in test:

    print(i)

Josh Hazlewood bowling to Quinton De Kock
EDGED AND GONE!
short of good length ball, pitching on leg stump, Quinton De Kock goes for his trademark cut but Hazlewood's magic seems to outdo QDK, inside edge and gloved safe behind the stumps. That's the first wicket of the game and RCB are elated
WICKET Quinton De Kock c Jitesh Sharma b Josh Hazlewood (Kolkata Knight Riders 4 Runs for 1 wickets)
It's all happening here ????
First wicket of
#TATAIPL
2025 ????????
And it goes to Josh Hazlewood ??
Updates ?
https://t.co/C9xIFpQ63P
#TATAIPL
|
#KKRvRCB
|
@RCBTweets
pic.twitter.com/ZRb8eujIHA
— IndianPremierLeague (@IPL)
March 22, 2025
